In [1]:
import sys
sys.getprofile()  # for linking sys
!{sys.executable} -m pip install -qUr requirements.txt

In [2]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_openai import OpenAIEmbeddings
from package import OPENAI_API_KEY, set_environment_openai_api_key
from package.utilities import get_file_contents
set_environment_openai_api_key()

chat = ChatOpenAI(model='gpt-4o', openai_api_key=OPENAI_API_KEY)
orchestrator_prompt = get_file_contents('orchestrator-prompt.txt')
entities = get_file_contents('entities.txt').splitlines()

In [3]:
aiMessage: AIMessage = chat.invoke([
    SystemMessage(content=orchestrator_prompt),
    HumanMessage(content="The manager of chat-bot's project has been changed, now it's Ivan Petrov")
])
candidates = [candidate.lstrip('0123456789.- ') for candidate in aiMessage.content.splitlines()]
print(candidates)

['manager', "chat-bot's project", 'Ivan Petrov']


In [4]:
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', headers={'x-api-key': OPENAI_API_KEY})
retriever = FAISS.from_texts(entities, embeddings).as_retriever()
for candidate in candidates:
    print(candidate, ' -> ', retriever.invoke(candidate)[0].page_content)

manager  ->  director
chat-bot's project  ->  bot program
Ivan Petrov  ->  person name


In [5]:
from langchain_community.vectorstores import Neo4jVector

title = 'title'

embeddings2 = OpenAIEmbeddings(model='text-embedding-ada-002', headers={'x-api-key': OPENAI_API_KEY})
vector_index = Neo4jVector.from_existing_graph(
    embeddings2,
    url='bolt://localhost:7687',
    username='neo4j',
    password='neo4j1234',
    index_name='node_text_index_title',
    node_label='Term',
    text_node_properties=[title],
    embedding_node_property='embedding',
)
for candidate in candidates:
    response = vector_index.similarity_search(
        f"Which `Term` is nearest for `{candidate}`?"
    )
    print(candidate, ' -> ', response[0].page_content.strip().removeprefix(f"{title}: "))

manager  ->  person
chat-bot's project  ->  bot program
Ivan Petrov  ->  neutron star


In [6]:
from uuid import uuid4
from peewee import *

db = SqliteDatabase('database.sqlite')

class BaseModel(Model):
    class Meta:
        database = db
        legacy_table_names = False

class Entity(BaseModel):
    id = UUIDField(primary_key=True, default=uuid4)
    type = CharField()  # entity title from Neo4j
    title = CharField()

class Relation(BaseModel):
    from_ = ForeignKeyField(Entity, backref='relations', db_column='from')
    to = ForeignKeyField(Entity, backref='relations', db_column='to')
    type = CharField()  # relation name from Neo4j

    class Meta:
        primary_key = CompositeKey('from_', 'to', 'type')

db.connect(True)
db.create_tables([Entity, Relation])
Entity.create_table()
# a = Entity.select().count()
# print(a)    

In [7]:
# Entity.create(title='New one', type='3db3c5bc-5431-425c-ae2a-00e4603a4578')
# Entity.create(title='Old two', type='4db3c5bc-5431-425c-ae2a-00e4603a4578')
# Relation.create(from_='3db3c5bc-5431-425c-ae2a-00e4603a4578', to='4db3c5bc-5431-425c-ae2a-00e4603a4578', type='Friend')
# Relation.create(from_='4db3c5bc-5431-425c-ae2a-00e4603a4578', to='4db3c5bc-5431-425c-ae2a-00e4603a4578', type='Friend')

In [11]:
from package.table_generator import *

attach_table_generator(db)

dataFoo = db.generate_table(Entity, 'foo1')
dataBar = db.generate_table(Relation, 'bar2')
